## Decision Tree Classifier
In this notebook we will implement decision tree classifier from scratch

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
class Node():
    def __init__(self, gini =0, num_samples=0, num_samples_per_class=None, predicted_class=None):
        self.gini = gini
        self.entropy  = 0
        self.left = None
        self.right = None
        self. predicted_class = predicted_class 
        self.num_sample_per_class = num_samples_per_class
        self.feature_index = 0
        self.threshold = 0

In [7]:
class DecisionTreeClassifier():
    def __init__(self,max_depth=20):
        self.max_depth = max_depth
    
    def fit(self,X,y):
        self.n_features = X.shape[1]
        self.num_classes = len(set(y))
        self.tree_root = self.grow_tree(X,y)

    
    def best_split(self,X,y):
        best_gain = 0.0
        best_feature_index = -1
        best_threshold = 0.0
        m = y.size
        if m<=1:
            return None,None,None
        
        for idx in range(self.n_features):
            
            feature = X[:,idx]
            thresholds = list(set(feature))
            
            for threshold in thresholds:
                gain = self.compute_gain(feature,y,threshold)
                if gain>0 and gain > best_gain:
                    best_gain = gain
                    best_feature_index = idx
                    best_threshold = threshold
                    
        return best_gain,best_feature_index,best_threshold
                    
        
    def grow_tree(self,X,y,depth=0):
        num_samples_per_class = [np.sum(y==i) for i in range(0,self.num_classes)]
        predicted_class = np.argmax(num_samples_per_class)
        gini = self.gini_impurity(y)
        node = Node(
            gini=gini,
            num_samples=y.size,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )
        if depth < self.max_depth:
            best_gain,idx, thr = self.best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] <= thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self.grow_tree(X_left, y_left, depth + 1)
                node.right = self.grow_tree(X_right, y_right, depth + 1)

        return node
        
        
    def gini_impurity(self,y):
        m = y.size
        p = sum(((np.sum(y==c)/m)**2) for c in range(self.num_classes))
        gini = 1.0 - p
        return gini
    
    
    def compute_gain(self,X,y,threshold):
        
        parent_gini = self.gini_impurity(y)
        
        left_y = y[X<=threshold]
        right_y = y[X>threshold]
        
        left_gini = self.gini_impurity(left_y)
        right_gini = self.gini_impurity(right_y)
        
        gain = parent_gini - (left_gini + right_gini)
        return gain
    
    
    def predict(self,X):
        
        predict_y = []
        for sample in X:
            current_node = self.tree_root
            while current_node.left:
                if sample[current_node.feature_index]<current_node.threshold:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
            predict_y.append(current_node.predicted_class)
        return predict_y

In [8]:
np.random.seed(21)
iris = load_iris()
x = iris.data
y = iris.target

decision_tree = DecisionTreeClassifier()
kf = KFold(n_splits=6, shuffle=True)

for i, (train, test) in enumerate(kf.split(x)):
    decision_tree.fit(x[train], y[train])
    accuracy = (decision_tree.predict(x[test]) == y[test]).sum() / len(y[test])
    print('{0}-validation accuracy: {1}'.format((i+1), accuracy))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in long_scalars


1-validation accuracy: 0.84
2-validation accuracy: 0.96
3-validation accuracy: 0.88
4-validation accuracy: 0.92
5-validation accuracy: 1.0
6-validation accuracy: 0.96
